# 支持横向联邦学习的 AutoModel

[上一节](10.%20AutoML%20%E6%9C%BA%E5%88%B6%E7%AE%80%E4%BB%8B.ipynb)介绍了本地模式的 AutoModel，这一节介绍如何设计支持横向联邦 FedAvg 算法的 AutoModel。要同时支持 AutoML 机制和 FedAvg 算法，其实就是要同时支持 `AutoModel` 和 `FedAvgScheduler` 两个接口。由于在一个实现类中同时实现二者的功能会使得这个类的逻辑过于复杂，不宜理解。因此在下面的示例中设计了两个实现类，分别对接两个接口，然后再通过二者之间的协作同时支持两个核心功能。

## 支持 FedAvg 算法的 AutoModel

在传统的联邦建模任务中，承担任务执行入口的是 `Scheduler` 对象。但是在联邦模式的自动建模任务中，承担执行微调入口的是 `AutoModel`。微调任务启动之后，需要通过联邦学习的方式完成微调。在接下来的示例中，采用的是 FedAvg 联邦学习算法，所以 `AutoModel` 需要为配合 `FedAvgScheduler` 做一些准备，然后通过启动 `FedAvgScheduler` 完成微调训练。为了方便实现这个目标，平台提供了 `AutoFedAvgModel` 基础类。通过 `AutoFedAvgModel` 基础类中定义的接口，实现模型设计逻辑与 FedAvg 联邦学习算法实现的隔离，便于开发者像设计本地模型一样设计 `AutoModel`。

### `AutoFedAvgModel` 基础类接口简介

In [ ]:
# 加载训练数据相关

from abc import abstractmethod
from torch.utils.data import DataLoader

# 以下三个接口均被设计为 @property 属性的形式。这是为了方便在训练过程中的不同地方得到同一个 DataLoader
# 对象，避免训练过程中由于疏忽而初始化并使用多个不同的 DataLoader 对象，导致数据混乱。
@property
@abstractmethod
def training_loader(self) -> DataLoader:
    """返回训练中使用的训练集数据的 DataLoader 对象。

    此接口必须实现。
    """

@property
@abstractmethod
def validation_loader(self) -> DataLoader:
    """返回训练中使用的验证集数据的 DataLoader 对象。

    此接口必须实现。
    """

@property
@abstractmethod
def testing_loader(self) -> DataLoader:
    """返回训练中使用的测试集数据的 DataLoader 对象。

    此接口必须实现。
    """

In [ ]:
# 核心模型相关

from typing import Optional
from torch import nn, optim

# 以下两个接口均被设计为 @property 属性的形式。原因与前面的三个 DataLoader 接口相同。
@property
@abstractmethod
def model(self) -> nn.Module:
    """返回训练中使用的核心模型对象.

    此接口必须实现。
    """

@property
@abstractmethod
def optimizer(self) -> optim.Optimizer:
    """返回训练中使用的核心模型优化器对象.

    此接口必须实现。
    """

In [ ]:
# 微调训练流程相关

from typing import Any, Dict
import torch

def state_dict(self) -> Dict[str, torch.Tensor]:
    """返回需要参与参数聚合的参数字典。

    此接口为可选择实现接口，默认返回模型的 state_dict()。如果需要定制参数更新方式，比如：
    锁定一部分模型参数只更新局部参数，或者聚合时包含优化器的参数，则需要自行修改实现逻辑。
    """
    return self.model.state_dict()

def load_state_dict(self, state_dict: Dict[str, torch.Tensor]):
    """state_dict 接口的逆操作，用于本地模型加载更新参数。

    参数说明:
        state_dict:
            更新使用的新模型参数。

    此接口为可选择实现接口，默认使用模型的 load_state_dict() 更新模型。
    如果需要定制参数更新方式，则需要自行修改实现逻辑。
    """
    self.model.load_state_dict(state_dict)

@abstractmethod
def train_an_epoch(self) -> Any:
    """模型本地训练时，训练一个 epoch 的代码逻辑。

    由于不同模型不同场景下的具体训练方式千差万别，所以需要算法工程师自行提供训练逻辑。

    此接口必须实现。
    """

@abstractmethod
def run_test(self) -> Any:
    """参数聚合后完成后，执行一次测试的代码逻辑。

    由于不同模型不同场景下的具体测试方式千差万别，所以需要算法工程师自行提供测试逻辑。

    此接口必须实现。
    """

def run_validation(self) -> Any:
    """执行一次验证的逻辑。

    此接口只在存在验证数据集时需要实现。
    """
    raise NotImplementedError()

def fine_tuned_files_dict(self) -> Optional[Dict[str, str]]:
    """返回微调后需要更新的模型加载文件映射字典。

    模型完成微调训练后，有一些初始加载文件的内容会发生变化。比如模型参数、config.json 文件中的类别标签信息等。
    当再次加载微调后的模型时，会希望加载新的初始化文件内容。这个接口提供了一个配置方式，可以在训练完成后打包模型
    文件压缩包时，按照映射关系将新的初始化文件加入压缩包的指定路径下，以用于加载微调后的新模型。

    配置格式示例:
    {
        '压缩包中的相对路径': '训练后新文件的实际路径'，
        'model.pt': 'TASK_ROOT_DIR/RESULT_DIR/model.pt'
    }

    此接口为可选择实现接口，默认返回空结果。
    """
    return {}

In [ ]:
# 工具方法，这里的方法是提供给开发者使用的，不需要也不应当自行实现或修改

from alphafed.auto_ml.auto_model import AutoModelError

@property
def result_dir(self):
    """返回当前任务保存训练结果的文件目录地址。

    为了安全原因，训练中使用的文件系统是受控的，不允许随意创建文件夹写数据。任务管理器为每一个任务规划了可用的文件
    目录，只有在这个目录下的文件才能够保证被正确处理。显然不应该让开发者自行去学习这个规则然后自己管理目录，所以由
    平台提供这个方法直接获取正确的目录，开发者只需要直接使用即可。
    """
    if hasattr(self, 'scheduler'):
        # self.scheduler 是与 AutoFedAvgModel 配合的 AutoFedAvgScheduler 调度器示例，后面介绍。
        return self.scheduler._result_dir
    else:
        raise AutoModelError('Can not save result files before initializing a scheduler.')

def push_log(self, message: str):
    """将日志推送至 Playground 前台的工具。

    默认的 logger 都是设计为在进程本地环境记录日志的，Playground 前台日志窗口看不到。当需要将日志信息推送到
    Playground 前台窗口时，可以使用这个工具接口。
    """
    # self.scheduler 是与 AutoFedAvgModel 配合的 AutoFedAvgScheduler 调度器示例，后面介绍。
    return self.scheduler.push_log(message)

### `AutoFedAvgModel` 基础类实现

现在定义一个继承了 `AutoFedAvgModel` 基础类的子类，实现基础类中定义的接口。示例中的实现类为 `AutoResNetFedAvg`。`AutoFedAvgModel` 首先也是一个 `AutoModel`，所以前面介绍的有关 `AutoModel` 的接口与配套实现同样适用于 `AutoFedAvgModel`。但是 `fine_tune` 接口例外，因为联邦模式下的训练是要通过联邦算法调度器实现的，而不能仅在本地完成。和本地模式 `AutoModel` 相同的内容就不重复介绍了，`fine_tune` 接口留在后面详细说明，这样需要说明的新接口就只有 `fine_tuned_files_dict` 一个了。

In [ ]:
from dataclasses import asdict
import json
import os
from alphafed.auto_ml.auto_model import AutoFedAvgModel

class AutoResNetFedAvg(AutoFedAvgModel):

    def fine_tuned_files_dict(self) -> Optional[Dict[str, str]]:
        # 保存新的模型参数文件
        param_file = os.path.join(self.result_dir, self.config.param_file)
        with open(param_file, 'wb') as f:
            torch.save(self.scheduler.best_state_dict, f)
        # 保存新的 config.json 配置文件
        config_file = os.path.join(self.result_dir, 'config.json')
        with open(config_file, 'w') as f:
            f.write(json.dumps(asdict(self.config), ensure_ascii=False))
        # 返回替换两个文件的映射
        return {
            self.config.param_file: param_file,
            'config.json': config_file
        }

对于 `fine_tune` 接口的实现，从流程上可以划分为微调训练前的准备和实际执行微调训练直至完成两个阶段。`AutoFedAvgModel` 准备阶段的工作与本地模式的 `AutoModel` 基本一致，但是运行阶段则改为调用 `_fine_tune_impl` 方法。`_fine_tune_impl` 方法封装了 FedAvg 联邦训练的控制逻辑，从而使得设计者无需再关心联邦学习与本地模式之间的细节差异，可以像设计本地模型一样设计 FedAvg 联邦模型训练。

In [ ]:
from res.auto_model_fed_avg.auto_fed_avg import ResNetFedAvgScheduler

class AutoResNetFedAvg(AutoFedAvgModel):

    def fine_tune(self,
                  id: str,
                  task_id: str,
                  dataset_dir: str,
                  is_initiator: bool = False,
                  recover: bool = False):
        self.id = id
        self.task_id = task_id
        self.is_initiator = is_initiator

        is_succ, err_msg = self.init_dataset(dataset_dir)
        if not is_succ:
            raise AutoModelError(f'Failed to initialize dataset. {err_msg}')
        num_classes = (len(self.training_loader.dataset.labels)
                       if self.training_loader
                       else len(self.testing_loader.dataset.labels))
        self._replace_fc_if_diff(num_classes)

        self.config.id2label = {str(_idx): _label for _idx, _label in enumerate(self.labels)}
        self.config.label2id = {_label: _idx for _idx, _label in enumerate(self.labels)}
        self.config.label2id = dict(sorted(self.config.label2id.items()))

        # 这部分是本地模式的训练逻辑，将其整体替换为 _fine_tune_impl
        # is_finished = False
        # self._epoch = 0
        # while not is_finished:
        #     self._epoch += 1
        #     self.push_log(f'Begin training of epoch {self._epoch}.')
        #     self._train_an_epoch()
        #     self.push_log(f'Complete training of epoch {self._epoch}.')
        #     is_finished = self._is_finished()

        # self._save_fine_tuned()
        # avg_loss, correct_rate = self._run_test()
        # self.push_log('\n'.join(('Testing result:',
        #                          f'avg_loss={avg_loss:.4f}',
        #                          f'correct_rate={correct_rate:.2f}')))
        # 本地模式的训练逻辑结束

        # AutoFedAvgModel 的训练方式
        self._fine_tune_impl(id=id,
                             task_id=task_id,
                             dataset_dir=dataset_dir,
                             # ResNetFedAvgScheduler 是配套的 FedAvgScheduler 实现，后面介绍
                             scheduler_impl=ResNetFedAvgScheduler,
                             is_initiator=is_initiator,
                             recover=recover,
                             max_rounds=self.config.epochs,
                             log_rounds=1)

至此，`AutoResNetFedAvg` 的实现就算完成了，完整代码可以看[这里](res/auto_model_fed_avg/auto_fed_avg.py)。

### `AutoFedAvgScheduler` 基础类实现

`AutoFedAvgScheduler` 的大部分作用是为 `FedAvgScheduler` 提供默认实现，从而帮助设计者摆脱 FedAvg 算法细节，只关注原始数据加载和核心模型优化，像设计本地模型一样设计 FedAvg 联邦模型训练。`AutoFedAvgScheduler` 的接口列表如下：

In [ ]:
from typing import Tuple
from zipfile import ZipFile
from alphafed.fed_avg import FedAvgScheduler
from alphafed.fed_avg.contractor import AutoFedAvgContractor

class AutoFedAvgScheduler(FedAvgScheduler):

    # self.auto_proxy 为初始化时传入的配套 AutoFedAvgModel 对象，下同。
    def build_model(self) -> nn.Module:
        return self.auto_proxy.model

    def build_optimizer(self, model: nn.Module) -> optim.Optimizer:
        return self.auto_proxy.optimizer

    def build_train_dataloader(self) -> DataLoader:
        return self.auto_proxy.training_loader

    def build_validation_dataloader(self) -> DataLoader:
        return self.auto_proxy.validation_loader

    def build_test_dataloader(self) -> DataLoader:
        return self.auto_proxy.testing_loader

    def state_dict(self) -> Dict[str, torch.Tensor]:
        return self.auto_proxy.state_dict()

    def load_state_dict(self, state_dict: Dict[str, torch.Tensor]):
        return self.auto_proxy.load_state_dict(state_dict)

    @property
    def best_state_dict(self) -> Dict[str, torch.Tensor]:
        """返回训练过程中得到的最优模型参数。

        在一般情况下，最优模型参数就是当前最新训练轮次的模型参数。但是在使用类似早停等控制机制的情况下，则会
        出现二者不一致的情况，比如选择验证集结果最优的那个轮次。

        此接口为可选实现，默认返回当前最新模型参数。
        """
        return self.state_dict()

除 `best_state_dict` 接口外，其余接口只需直接返回 `AutoFedAvgModel` 的对应接口调用。示例中定义了 `ResNetFedAvgScheduler` 继承 `AutoFedAvgScheduler` 基础类，`ResNetFedAvgScheduler` 的完整代码参考[这里](res/auto_model_fed_avg/auto_fed_avg.py)。`ResNetFedAvgScheduler` 的主要改动是重新定义了 `is_task_finished` 接口逻辑，以支持早停算法，防止过拟合，同时配合 `best_state_dict` 接口记录返回最优模型参数。除此之外的几个接口修改都只是一些细微调整。

至此，`AutoFedAvgModel` 实现和 `AutoFedAvgScheduler` 实现都已准备就绪，可开始调试运行。

## 调试运行使用 FedAvg 横向联邦算法的 AutoModel

接下来将展示如何在模拟环境中调试 `AutoResNetFedAvg`。数据集继续使用[上一节](10.%20AutoML%20%E6%9C%BA%E5%88%B6%E7%AE%80%E4%BB%8B.ipynb)使用的 HAM10000 数据集。

首先需要打开 Notebook 调试环境，先测试一下 AutoModel 模型的加载。

In [ ]:
from alphafed.auto_ml import from_pretrained

auto_model = from_pretrained(resource_dir='res/auto_model_fed_avg/')

如果模型初始化成功，下一步试试能否正确加载训练数据。

In [ ]:
is_succ, help_text = auto_model.init_dataset(dataset_dir='res/data/HAM10000')
print(f'数据是否加载成功: {is_succ}')
print(f'提示信息: {help_text}')
if is_succ:
    print(f'包含训练集样本: {len(auto_model.training_loader.dataset)}')
    print(f'包含验证集样本: {len(auto_model.validation_loader.dataset)}')
    print(f'包含测试集样本: {len(auto_model.testing_loader.dataset)}')

现在准备测试一下微调的效果。分别准备三个 Notebook 脚本文件模拟三个参与方。简单起见聚合节点仅做参数聚合，不参与本地模型训练。三个参与方的模拟脚本如下：

In [ ]:
# 聚合方的模拟启动脚本

from alphafed import mock_context
from alphafed.auto_ml import from_pretrained

auto_model = from_pretrained(resource_dir='auto_model_fed_avg')

task_id = '11c12dc5-0473-4932-930e-ad56c69c5ea1'  # 必须与聚合方配置相同
aggregator_id = 'd5f978fa-84f5-4724-b4f5-8abb317be4e2'  # 必须与聚合方配置相同
col_id_1 = '4d43ea09-aad6-4beb-bc23-105e90ad5567'   # 必须与聚合方配置相同
col_id_2 = 'ff2ce0a2-6983-45d6-8512-151e71710928'  # 必须与聚合方配置相同
with mock_context(id=aggregator_id, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    auto_model.fine_tune(id=aggregator_id,
                         task_id=task_id,
                         dataset_dir='data/HAM10000',
                         is_initiator=True)


# 参与方的模拟启动脚本，需要复制到另一个 Notebook 脚本文件中执行
auto_model = from_pretrained(resource_dir='auto_model_fed_avg')

task_id = '11c12dc5-0473-4932-930e-ad56c69c5ea1'  # 必须与聚合方配置相同
aggregator_id = 'd5f978fa-84f5-4724-b4f5-8abb317be4e2'  # 必须与聚合方配置相同
col_id_1 = '4d43ea09-aad6-4beb-bc23-105e90ad5567'   # 必须与聚合方配置相同
col_id_2 = 'ff2ce0a2-6983-45d6-8512-151e71710928'  # 必须与聚合方配置相同
with mock_context(id=col_id_1, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    auto_model.fine_tune(id=col_id_1,
                         task_id=task_id,
                         dataset_dir='data/HAM10000',
                         is_initiator=False)


# 另一个参与方的模拟启动脚本，需要复制到另一个 Notebook 脚本文件中执行
auto_model = from_pretrained(resource_dir='auto_model_fed_avg')

task_id = '11c12dc5-0473-4932-930e-ad56c69c5ea1'  # 必须与聚合方配置相同
aggregator_id = 'd5f978fa-84f5-4724-b4f5-8abb317be4e2'  # 必须与聚合方配置相同
col_id_1 = '4d43ea09-aad6-4beb-bc23-105e90ad5567'   # 必须与聚合方配置相同
col_id_2 = 'ff2ce0a2-6983-45d6-8512-151e71710928'  # 必须与聚合方配置相同
with mock_context(id=col_id_2, nodes=[aggregator_id, col_id_1, col_id_2]):  # 在模拟调试环境中运行
    auto_model.fine_tune(id=col_id_1,
                         task_id=task_id,
                         dataset_dir='data/HAM10000',
                         is_initiator=False)

整理好的[聚合方脚本](res/11_aggregator.ipynb)、[参与方-1 脚本](res/11_collaborator_1.ipynb)、[参与方-2 脚本](res/11_collaborator_2.ipynb)均可以直接运行。